In [1]:
include("../LiPoSID.jl")

Main.LiPoSID

In [7]:
#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]
γ = [ "25.133", "79.477", "251.33"]

γᵢ  = γ[1]
γᵢ = string(γᵢ)

data_dir = ""

ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B1_data.h5", γᵢ)
ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B2_data.h5", γᵢ)
ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_data.h5", γᵢ)
ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_data.h5", γᵢ)

ρ⁵, t⁵ = LiPoSID.get_rho_series(data_dir*"State_B5_data.h5", γᵢ)
ρ⁶, t⁶ = LiPoSID.get_rho_series(data_dir*"State_B6_data.h5", γᵢ);

$\rho = k_e \rho_e + k_g \rho_g + k_x \rho_x + k_y \rho_y$

$\operatorname{vec}[\rho] = k_e \operatorname{vec}[\rho_e] + k_g \operatorname{vec}[\rho_g] + k_x \operatorname{vec}[\rho_x] + k_y \operatorname{vec}[\rho_y]$

$\vec{\rho} = \begin{pmatrix} | \\ \rho \\ |\end{pmatrix} = \begin{pmatrix} | & | & | & | \\\rho_e & \rho_g & \rho_x & \rho_y \\ | & | & | & | \\ \end{pmatrix} \begin{pmatrix} k_e \\ k_g \\ k_x \\  k_y \end{pmatrix} = P \vec{k}$

$\vec{k} = P^+ \vec{\rho}$


In [10]:
k = hcat(vec(ρᵉ[1]), vec(ρᵍ[1]), vec(ρˣ[1]), vec(ρʸ[1]))\vec(ρ⁵[1])

4-element Vector{ComplexF64}:
  1.0 + 0.0im
  1.0 + 0.0im
 -1.0 + 0.0im
  0.0 + 0.0im

In [41]:
ρᵉᵍˣʸ = [ρᵉ, ρᵍ, ρˣ, ρʸ]

4-element Vector{Vector{Any}}:
 [ComplexF64[1.0 + 0.0im 0.0 - 0.0im; 0.0 + 0.0im 0.0 + 0.0im], ComplexF64[0.999979900097 + 0.0im 0.0 - 0.0im; 0.0 + 0.0im 2.009990300000375e-5 + 0.0im], ComplexF64[0.99991967546 + 0.0im 0.0 - 0.0im; 0.0 + 0.0im 8.032453999995415e-5 + 0.0im], ComplexF64[0.999819550787 + 0.0im 0.0 - 0.0im; 0.0 + 0.0im 0.00018044921299997796 + 0.0im], ComplexF64[0.999679898846 + 0.0im 0.0 - 0.0im; 0.0 + 0.0im 0.000320101154000052 + 0.0im], ComplexF64[0.999501237888 + 0.0im 0.0 - 0.0im; 0.0 + 0.0im 0.0004987621119999508 + 0.0im], ComplexF64[0.999284228075 + 0.0im 0.0 - 0.0im; 0.0 + 0.0im 0.0007157719250000527 + 0.0im], ComplexF64[0.999029666938 + 0.0im 0.0 - 0.0im; 0.0 + 0.0im 0.0009703330620000061 + 0.0im], ComplexF64[0.998738483921 + 0.0im 0.0 - 0.0im; 0.0 + 0.0im 0.0012615160790000468 + 0.0im], ComplexF64[0.998411734046 + 0.0im 0.0 - 0.0im; 0.0 + 0.0im 0.0015882659539999677 + 0.0im]  …  ComplexF64[0.803946745753 + 0.0im -1.896e-9 + 1.6251e-8im; -1.896e-9 - 1.6251e-8im 0.1

In [38]:
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra

function min_fidelity_linearity_check(ρᵉᵍˣʸ, ρ)
    k = hcat([vec(ρᵢ[1]) for ρᵢ in ρᵉᵍˣʸ]...)\vec(ρ[1])
    l=minimum(length.(ρᵉᵍˣʸ))
    ρˡⁱⁿ = reduce(+, [k[i] * ρᵉᵍˣʸ[i][1:l] for i in 1:length(k)])
    ρˡⁱⁿ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρˡⁱⁿ]
    ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρ]
    Fˡⁱⁿ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˡⁱⁿ)] 
    return(minimum(Fˡⁱⁿ))
end

min_fidelity_linearity_check (generic function with 1 method)

In [39]:
min_fidelity_linearity_check(ρᵉᵍˣʸ, ρ⁵)

0.9999999971912648

In [49]:
1-min_fidelity_linearity_check(ρᵉᵍˣʸ, ρ⁶)

1.7627892479055163e-9

In [42]:
function generate_by_linear_composition(ρᵉᵍˣʸ, ρ₀)
    k = hcat([vec(ρᵢ[1]) for ρᵢ in ρᵉᵍˣʸ]...)\vec(ρ₀)
    ρˡⁱⁿ = reduce(+, [k[i] * ρᵉᵍˣʸ[i][1:l] for i in 1:length(k)])
end

generate_by_linear_composition (generic function with 1 method)

In [48]:
maximum(maximum(abs.(ρ)) for ρ in (ρ⁶ - generate_by_linear_composition(ρᵉᵍˣʸ, ρ⁶[1])))

4.933674299989477e-5